In [1]:
#check model
%load_ext autoreload
%load_ext line_profiler
%autoreload 2

import numpy as np
import pandas as pd
from mpl_toolkits import mplot3d
import matplotlib
import matplotlib.pyplot as plt
import scipy.optimize as optimize
import Estimate
import figure 
from EconModel import cpptools
import pickle

from LimitedCommitmentModel import LimitedCommitmentModelClass

# plot style
linestyles = ['-','--','-.',':',':']
markers = ['o','s','D','*','P']
linewidth = 2
font_size = 17
font = {'size':font_size}
matplotlib.rc('font', **font)


plt.rcParams.update({'figure.max_open_warning': 0,'text.usetex': False})
path = 'output/'

SAVE = False

# c++ settings

do_compile = True
threads = 20
T = 20
#cpptools.setup_nlopt(folder='cppfuncs/', do_print = True) #install nlopt

#SOLVE THE MODEL

In [2]:
save_data = False
save_pickle = True
specs = {}

#JEG PRØVER AT KØRE MED 0.25 ved util Z


#specs.update({f'model_NC':{'latexname':'limited', 'par':{ 'T':T, 'do_HK': False, 'threads':threads,'bargaining':2}}})
specs.update({f'model_FC':{'latexname':'limited', 'par':{ 'T':T,  'do_HK': False, 'threads':threads,'bargaining':0}}})
#specs.update({f'model_LC':{'latexname':'limited', 'par':{ 'T':T, 'do_HK':False , 'threads':threads,'bargaining':1}}})

#specs.update({f'model_FC_doHK':{'latexname':'limited', 'par':{ 'T':T,  'do_HK': True, 'threads':threads,'bargaining':0}}})
#specs.update({f'model_LC_doHK':{'latexname':'limited', 'par':{ 'T':T,  'do_HK': True , 'threads':threads,'bargaining':1}}}) 
#specs.update({f'model_NC_doHK':{'latexname':'limited', 'par':{ 'T':T, 'do_HK': True, 'threads':threads,'bargaining':2}}})

print(specs)



# solve different models
models = {}
for m,(name,spec) in enumerate(specs.items()):
    print(f'{name} loading...',end='')
    
    # setup model
    models = LimitedCommitmentModelClass(name=name,par=spec['par'])
    models.spec = spec

    compile_now = True if do_compile & (m==0) else False
    models.link_to_cpp(force_compile=compile_now)
    
    print(' solving...')
    %time models.solve() 

    if save_pickle:
        pickle_file = f'output/{name}_sol.pickle'
        with open(pickle_file, 'wb') as f:
            pickle.dump(models.sol,f)
        pickle_file = f'output/{name}_par.pickle'
        with open(pickle_file, 'wb') as f:
            pickle.dump(models.par,f)

    #Save  data 
    if save_data: 
        T = models.par.T
        print(' saving...')
        models.sim.init_love[:] = 0.2
        models.sim.init_A[:] = 2.0 
        np.random.seed(models.par.seed)
        data1 = Estimate.create_data(models,start_p = 1, end_p = T-1, to_xl = True, name_xl = name)
        print(f'Couple {np.mean(models.sim.couple,0)}')
        print(f'HKw {np.mean(models.sim.Kw,0)}')
        print(f'HKm {np.mean(models.sim.Km,0)}')
        print(f'Laborw {np.mean(models.sim.labor_w,0)}')
        print(f'Laborm {np.mean(models.sim.labor_m,0)}')
        print(f'Asset {np.nanmean(models.sim.A,0)}')

        var_list = ('labor_w', 'labor_m', 'cons_w', 'cons_m', 'Vw', 'Vm')
        t_list = (1,9)
        i_A = 250
        i_HK = 1
        i_HKw = i_HK
        i_HKm = i_HK
        i_Z = 0
        i_Zw = i_Z
        i_Zm = i_Z
        i_P = 5
        i_L = 5
        figure.figure_single(models, var_list, t_list, i_A, i_Z, i_HK, model_name = name)
        figure.figure_couple(models, var_list, t_list, i_A, i_Zw, i_Zm, i_HKw, i_HKm, i_L, i_P, sol_type = 'couple', model_name = name)
        figure.figure_couple(models, var_list, t_list, i_A, i_Zw, i_Zm, i_HKw, i_HKm, i_L, i_P, sol_type = 'remain_couple', model_name = name)


{'model_FC': {'latexname': 'limited', 'par': {'T': 20, 'do_HK': False, 'threads': 20, 'bargaining': 0}}}
model_FC loading... solving...
CPU times: total: 23h 41min 29s
Wall time: 2h 54min 10s


NameError: name 'pickle_file' is not defined

In [ ]:

stop

In [3]:
name = "model_FC"
pickle_file = f'output/{name}_sol.pickle'
with open(pickle_file, 'wb') as f:
    pickle.dump(models.sol,f)
pickle_file = f'output/{name}_par.pickle'
with open(pickle_file, 'wb') as f:
    pickle.dump(models.par,f)


NameError: name 'model_FC' is not defined